# 07 — Facet Comparison: Blocking vs Persistent Ridge (PRP)

Compares the PV-budget decomposition between **blocking** and **PRP** events:

1. Composite-mean basis and coefficients for both event types
2. Lifecycle coefficient curves side by side
3. Grouped-term decomposition comparison

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob
from concurrent.futures import ThreadPoolExecutor

from pvtend import compute_orthogonal_basis, project_field
from pvtend.decomposition.smoothing import gaussian_smooth_nan

## 1  Helper functions

In [ ]:
DATA_ROOT = "/net/flood/data2/users/x_yan/tempest_extreme_4_basis/outputs_tmp"

def load_all_events(event_type, stage, dh):
    """Load all NPZ files for a given event type / stage / dh."""
    sign = "+" if dh >= 0 else ""
    subdir = "blocking_tmp" if event_type == "blocking" else "prp_tmp"
    npz_dir = f"{DATA_ROOT}/{subdir}/{stage}/dh={sign}{dh}"
    files = sorted(glob.glob(os.path.join(npz_dir, "track_*.npz")))

    def _load(f):
        return dict(np.load(f))

    with ThreadPoolExecutor(max_workers=8) as pool:
        return list(pool.map(_load, files))

def compute_composite_basis(events, x_rel, y_rel):
    """Build orthogonal basis from composite-mean PV fields."""
    pv_mean  = np.mean([e["pv_anom"]    for e in events], axis=0)
    dx_mean  = np.mean([e["pv_anom_dx"] for e in events], axis=0)
    dy_mean  = np.mean([e["pv_anom_dy"] for e in events], axis=0)
    return compute_orthogonal_basis(
        pv_mean, dx_mean, dy_mean, x_rel, y_rel,
        mask_negative=True, apply_smoothing=True,
        smoothing_deg=6.0, grid_spacing=1.5,
    )

GROUPS = {
    "leading_order_adv": [
        "u_bar_pv_anom_dx", "v_bar_pv_anom_dy",
        "u_anom_pv_bar_dx", "v_anom_pv_bar_dy",
    ],
    "eddy_rot": [
        "u_rot_pv_anom_dx", "v_rot_pv_anom_dy",
    ],
    "eddy_div": [
        "u_div_pv_anom_dx", "v_div_pv_anom_dy",
    ],
}

def sum_group(event, terms):
    total = np.zeros_like(event["pv_anom"])
    for t in terms:
        if t in event:
            total -= event[t]   # sign convention: -u·∇q
    return total

## 2  Load events and compute lifecycle curves

In [ ]:
STAGE = "onset"
dh_values = list(range(-13, 13))
event_types = ["blocking", "prp"]

results = {}
for etype in event_types:
    print(f"\nProcessing {etype}...")
    coefs_dh = {g: {k: [] for k in ["beta", "ax", "ay", "gamma"]}
                for g in list(GROUPS.keys()) + ["pv_dt"]}

    for dh in dh_values:
        events = load_all_events(etype, STAGE, dh)
        if not events:
            for g in coefs_dh:
                for k in coefs_dh[g]:
                    coefs_dh[g][k].append(np.nan)
            continue

        x_rel = events[0]["X_rel"][0, :]
        y_rel = events[0]["Y_rel"][:, 0]

        # Build basis at dh-1
        dh_basis = max(dh - 1, -13)
        events_basis = load_all_events(etype, STAGE, dh_basis)
        basis = compute_composite_basis(events_basis, x_rel, y_rel)

        # Project pv_dt
        pv_dt_mean = np.mean([e["pv_anom_dt"] for e in events], axis=0)
        pv_dt_s = gaussian_smooth_nan(pv_dt_mean, smoothing_deg=6.0, grid_spacing=1.5)
        p = project_field(pv_dt_s, basis)
        for k in ["beta", "ax", "ay", "gamma"]:
            coefs_dh["pv_dt"][k].append(p[k])

        # Project grouped terms
        for gname, terms in GROUPS.items():
            gmean = np.mean([sum_group(e, terms) for e in events], axis=0)
            gmean_s = gaussian_smooth_nan(gmean, smoothing_deg=6.0, grid_spacing=1.5)
            p = project_field(gmean_s, basis)
            for k in ["beta", "ax", "ay", "gamma"]:
                coefs_dh[gname][k].append(p[k])

        print(f"  dh={dh:+3d}  N={len(events)}  β(pv_dt)={coefs_dh['pv_dt']['beta'][-1]:.3e}")

    # Convert to arrays
    for g in coefs_dh:
        for k in coefs_dh[g]:
            coefs_dh[g][k] = np.array(coefs_dh[g][k])

    results[etype] = coefs_dh

## 3  Facet plot: blocking vs PRP lifecycle curves

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(16, 9))
coef_keys = ["beta", "ax", "ay"]
coef_labels = ["β  [s⁻¹]", "αx  [m/s]", "αy  [m/s]"]
dh_arr = np.array(dh_values)

# Row 0: leading_order_adv
# Row 1: eddy_rot
row_groups = ["leading_order_adv", "eddy_rot"]
row_titles = ["Leading-order advection", "Eddy (rotational)"]

for row, gname in enumerate(row_groups):
    for col, (ckey, clabel) in enumerate(zip(coef_keys, coef_labels)):
        ax = axes[row, col]
        for etype, ls in [("blocking", "-"), ("prp", "--")]:
            ax.plot(dh_arr, results[etype][gname][ckey], ls,
                    label=etype.capitalize(), lw=2)
        ax.axhline(0, color="k", lw=0.5, ls=":")
        ax.axvline(0, color="gray", lw=0.5, ls=":")
        ax.set_xlabel("dh [hours]")
        if col == 0:
            ax.set_ylabel(f"{row_titles[row]}\n{clabel}")
        else:
            ax.set_ylabel(clabel)
        if row == 0 and col == 0:
            ax.legend(fontsize=9)

fig.suptitle(f"Blocking vs PRP — {STAGE} lifecycle", fontsize=14, y=1.01)
fig.tight_layout()
plt.show()

## 4  Bar comparison at dh = 0

In [ ]:
idx0 = dh_values.index(0)

fig, axes = plt.subplots(1, 3, figsize=(14, 5))
group_names = list(GROUPS.keys())
x_pos = np.arange(len(group_names))
width = 0.35

for ax, ckey, clabel in zip(axes, coef_keys, coef_labels):
    vals_b = [results["blocking"][g][ckey][idx0] for g in group_names]
    vals_p = [results["prp"][g][ckey][idx0] for g in group_names]

    ax.bar(x_pos - width/2, vals_b, width, label="Blocking", color="steelblue")
    ax.bar(x_pos + width/2, vals_p, width, label="PRP", color="coral")
    ax.set_xticks(x_pos)
    ax.set_xticklabels([g.replace("_", "\n") for g in group_names], fontsize=8)
    ax.set_ylabel(clabel)
    ax.axhline(0, color="k", lw=0.5)
    if ax == axes[0]:
        ax.legend()

fig.suptitle(f"Grouped-term coefficients at {STAGE} dh=0", fontsize=13, y=1.02)
fig.tight_layout()
plt.show()

## Summary

- **Blocking** and **PRP** events share similar leading-order advective
  contributions but differ systematically in the eddy (nonlinear) terms.
- The rotational eddy term drives **intensification** (positive β) in blocking
  but is weaker or opposite-signed for PRP.
- The **lifecycle curves** show how these differences evolve from pre-onset
  through decay, with blocking maintaining stronger eddy forcing.